# Mashup query
projekt grupy:
* Bartosz Mierzwa
* Pawel Król
* Tymoteusz Białkowski
* Rodosław Szydłowski

## 0. Przygotowania mashupu

### Instalowanie biblitek

In [1]:
!pip install sparqlwrapper rdflib owlready2

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import SPARQLWrapper
import urllib
import rdflib
import requests
import json
import pandas as pd
from IPython.display import display

### Wycztywanie danych
Wczytywanie danych z pliku csv.

In [3]:
# link do danych w formacie csv
url = 'https://sharing.srv39249.seohost.com.pl/semantyczny/drivers.csv'

drivers_df = pd.read_csv(url)
drivers_df.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


### Przygotowanie zapytań

In [4]:
dbpedia = SPARQLWrapper.SPARQLWrapper('https://dbpedia.org/sparql')

prefixes = f"""
  PREFIX dbo: <http://dbpedia.org/ontology/>
  PREFIX dbp: <http://dbpedia.org/property/>
  PREFIX dbr: <http://dbpedia.org/resource/>
  PREFIX foaf: <http://xmlns.com/foaf/0.1/>
"""

prop = ['', 'dbo:firstDriver', 'dbo:secondDriver', 'dbo:thirdDriver']
place_str = ['0th', '1st', '2nd', '3rd']

def getDriverInfoQueryString(urls):
  urls = ' '.join('<{}>'.format(s.strip()) for s in urls)
  return f""" {prefixes}
    SELECT ?birthName, ?birthPlace, ?url WHERE {{
      ?driver a dbo:FormulaOneRacer ;
              foaf:isPrimaryTopicOf ?url ;
              dbp:birthPlace ?birthPlace ;
              dbp:birthName ?birthName .
    FILTER (lang(?birthPlace) = "en")
    FILTER (lang(?birthName) = "en")
    VALUES ?url {{{urls}}}
    }}
    """



def getDriverAchivemntsQueryString(url, place):
  return f""" {prefixes}
  SELECT ?raceName, ?year WHERE {{
      ?driver a dbo:FormulaOneRacer ;
              foaf:isPrimaryTopicOf <{url}> .
      ?race {prop[place]} ?driver ;
          dbp:year ?year ;
          dbp:nameOfRace ?race1 .
      ?race1 rdfs:label ?raceName .
    FILTER (lang(?raceName) = "en")
    }}
    """

def queryDBpedia(query):
  dbpedia.setQuery(query)
  dbpedia.setReturnFormat(SPARQLWrapper.JSON)
  return dbpedia.query().convert()

def getTabularData(url, place):
	res = queryDBpedia(getDriverAchivemntsQueryString(url, place))
	tmp = []
	for x in res['results']['bindings']:
		tmp.append({place_str[place]: x['raceName']['value'], 'year': x['year']['value']})
	return pd.DataFrame(tmp).set_index(['year']).groupby("year").count()

## 1. mashup query - dane kierowcy
Wyświetla dane o kierowcy z pliku csv oraz z dbpedii. Imię i nazwisko nalezy wprowadzić na wejściu oddzielając je przecinkiem. Można podać imiona i nazwiska wielu kierowców, oddzielając je przecinkami. Zapytanie wypisze dane dla wszystkich kierowców, dla których udało się znaleść informacje.
Przykłady:
* Robert Kubica 
* Lewis Hamilton
* Robert Kubica, Lewis Hamilton
* Robert Kubica, Lewis Hamilton, Nico Rosberg, Fernando Alonso

In [9]:
name_input = input('Wprowadź imię i nazwisko kierowcy (na przykład "Lewis Hamilton", wiele oddziel przecinkiem): ')
name_input = [s.strip() for s in name_input.split(',')]

driver_df = drivers_df[(drivers_df['forename'] + ' ' + drivers_df['surname']).isin(name_input)]
driver_df = driver_df.drop(['driverRef', 'driverId'], axis='columns')\
                     .rename({'dob': 'birth date'}, axis='columns')\
                     .reset_index()
urls = driver_df['url'].values

resInfo = queryDBpedia(getDriverInfoQueryString(urls))
birthNames = [res['birthName']['value'] for res in resInfo['results']['bindings']]
birthPlaces = [res['birthPlace']['value'] for res in resInfo['results']['bindings']]
urls = [res['url']['value'] for res in resInfo['results']['bindings']]
temp_df = pd.DataFrame({'birth name': birthNames, 'birth places': birthPlaces, 'url': urls})
driver_df = driver_df.merge(temp_df, on='url')\
                     .drop(['index'], axis='columns')

display(driver_df)

,number,code,forename,surname,birth date,nationality,url,birth name,birth places
0,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis Carl Davidson Hamilton,"Stevenage, Hertfordshire, England"


## 2. mashup query - wyniki kierowcy
Wygeneruj tabelę, dla której mashup zliczy ilość wyścigów w danym roku, w których kierowca osiągną pierwsze, drugie lub trzecie miejsce.
W pierwszym polu wprowadź poprawne imię i nazwisko kierowcy oddzielone spacją Przykłady:
* Lewis Hamilton
* Robert Kubica

W drugim polu wprowadź miejsce na podium, dla którego statystyki mają zostac policzone. Przykłady:

* 1
* 1,3
* 1,2,3

In [10]:
name_input = input('Wprowadź imię i nazwisko kierowcy (na przykład "Lewis Hamilton"): ')
place_input = input('Wprwadź miejsca na podium (1 - pierwsze, 2 - drugie, 3 -trzecie, wiele oddziel przecinkiem')
place_input = [int(x) for x in place_input.split(',')]

driver_df = drivers_df.loc[(drivers_df['forename'] == name_input.split()[0]) & (drivers_df['surname'] == name_input.split()[1])]
url = driver_df['url'].values[0]

df = getTabularData(url, place_input.pop())
for n in place_input:
    df = df.merge(getTabularData(url, n), on='year')
df.sort_values(by="year", ascending=False)

display(df)

,2nd,1st
year,,
2007,5,4
2008,2,5
2009,1,2
2010,5,3
2011,3,3
2014,3,11
2015,6,10
2016,3,10
2017,4,9
